# Memory

<div style="display: flex; justify-content: flex-start; gap: 10px;">
  <img src="./assets/LC_Memory_after.png" style="width:300px; border:1px solid #ccc; border-radius:6px;">
</div>

Persisting messages, or 'agent state' between invocations of the agent.

## Setup

Load and/or check for needed environmental variables

In [1]:
from dotenv import load_dotenv
from env_utils import doublecheck_env

# Load environment variables from .env
load_dotenv()

# Check and print results
doublecheck_env("example.env")

OPENAI_API_KEY=****here
DEEPSEEK_API_KEY=****ba4e
LANGSMITH_API_KEY=****here
LANGSMITH_TRACING=false
LANGSMITH_PROJECT=****ials


In [2]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")

In [3]:
from dataclasses import dataclass

@dataclass
class RuntimeContext:
    db: SQLDatabase

In [4]:
from langchain_core.tools import tool
from langgraph.runtime import get_runtime

@tool
def execute_sql(query: str) -> str:
    """Execute a SQLite command and return results."""
    runtime = get_runtime(RuntimeContext)
    db = runtime.context.db

    try:
        return db.run(query)
    except Exception as e:
        return f"Error: {e}"

In [5]:
SYSTEM_PROMPT = """You are a careful SQLite analyst.

Rules:
- Think step-by-step.
- When you need data, call the tool `execute_sql` with ONE SELECT query.
- Read-only only; no INSERT/UPDATE/DELETE/ALTER/DROP/CREATE/REPLACE/TRUNCATE.
- Limit to 5 rows unless the user explicitly asks otherwise.
- If the tool returns 'Error:', revise the SQL and try again.
- Prefer explicit column lists; avoid SELECT *.
"""

In [7]:
from langchain.agents import create_agent

agent = create_agent(
    model="deepseek:deepseek-chat",
    tools=[execute_sql],
    system_prompt=SYSTEM_PROMPT,
    context_schema=RuntimeContext,
)

## Repeated Queries

In [9]:
question = "This is Frank Harris, What was the total on my last invoice?"
steps = []

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    stream_mode="values",
    context=RuntimeContext(db=db),
):
    step["messages"][-1].pretty_print()
    steps.append(step)

================================ Human Message =================================

This is Frank Harris, What was the total on my last invoice?
================================== Ai Message ==================================

I'll help you find the total on Frank Harris's last invoice. Let me start by looking for Frank Harris in the database and then find his most recent invoice.
Tool Calls:
  execute_sql (call_00_oV3bmrwy9FIYLIP4YyFPdtNk)
 Call ID: call_00_oV3bmrwy9FIYLIP4YyFPdtNk
  Args:
    query: SELECT * FROM customers WHERE first_name = 'Frank' AND last_name = 'Harris'
================================= Tool Message =================================
Name: execute_sql

Error: (sqlite3.OperationalError) no such table: customers
[SQL: SELECT * FROM customers WHERE first_name = 'Frank' AND last_name = 'Harris']
(Background on this error at: https://sqlalche.me/e/20/e3q8)
================================== Ai Message ==================================

Let me first check what tables are

In [11]:
question = "What were the titles?"

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What were the titles?
================================== Ai Message ==================================

I'd be happy to help you find the titles, but I need a bit more context to know which titles you're looking for. Could you please specify:

1. Which table or database you're referring to?
2. What type of titles are you interested in (e.g., book titles, movie titles, article titles, etc.)?

If you know the table name, that would be most helpful. Otherwise, I can first check what tables are available in the database to help us locate the right information.


## Add memory

In [12]:
from langgraph.checkpoint.memory import InMemorySaver

In [13]:
from langchain.agents import create_agent
from langchain_core.messages import SystemMessage

agent = create_agent(
    model="deepseek:deepseek-chat",
    tools=[execute_sql],
    system_prompt=SYSTEM_PROMPT,
    context_schema=RuntimeContext,
    checkpointer=InMemorySaver(),
)

In [14]:
question = "This is Frank Harris, What was the total on my last invoice?"
steps = []

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    {"configurable": {"thread_id": "1"}},
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()
    steps.append(step)

================================ Human Message =================================

This is Frank Harris, What was the total on my last invoice?
================================== Ai Message ==================================

I'll help you find the total on Frank Harris's last invoice. Let me start by looking for Frank Harris in the database and then find his most recent invoice.
Tool Calls:
  execute_sql (call_00_zTDpYK620wto00yTSBI0WuIM)
 Call ID: call_00_zTDpYK620wto00yTSBI0WuIM
  Args:
    query: SELECT * FROM customers WHERE first_name = 'Frank' AND last_name = 'Harris'
================================= Tool Message =================================
Name: execute_sql

Error: (sqlite3.OperationalError) no such table: customers
[SQL: SELECT * FROM customers WHERE first_name = 'Frank' AND last_name = 'Harris']
(Background on this error at: https://sqlalche.me/e/20/e3q8)
================================== Ai Message ==================================

Let me first explore what tables a

In [15]:
question = "What were the titles?"
steps = []

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    {"configurable": {"thread_id": "1"}},
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()
    steps.append(step)

================================ Human Message =================================

What were the titles?
================================== Ai Message ==================================

I'll find the titles of the tracks on Frank Harris's last invoice (Invoice ID 374). Let me query the InvoiceLine and Track tables to get this information.
Tool Calls:
  execute_sql (call_00_BYXi5lnSgRb2szBOToCApqdV)
 Call ID: call_00_BYXi5lnSgRb2szBOToCApqdV
  Args:
    query: SELECT t.Name as TrackTitle, a.Name as ArtistName, t.UnitPrice 
FROM InvoiceLine il
JOIN Track t ON il.TrackId = t.TrackId
JOIN Album al ON t.AlbumId = al.AlbumId
JOIN Artist a ON al.ArtistId = a.ArtistId
WHERE il.InvoiceId = 374
================================= Tool Message =================================
Name: execute_sql

[('Holier Than Thou', 'Metallica', 0.99), ('Through The Never', 'Metallica', 0.99), ('My Friend Of Misery', 'Metallica', 0.99), ('The Wait', 'Metallica', 0.99), ('Blitzkrieg', 'Metallica', 0.99), ('So What'

## Try your own queries
Now that there is memory, check the agents recall!

In [19]:
question = "How mush is 'So What'?"
steps = []

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    {"configurable": {"thread_id": "1"}}, # 通过thread_id记住记忆
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()
    steps.append(step)

================================ Human Message =================================

How mush is 'So What'?
================================== Ai Message ==================================

I'll check the price of the track "So What" by Metallica from the database.
Tool Calls:
  execute_sql (call_00_Sw6k9N1nLRgji7URofe5gBom)
 Call ID: call_00_Sw6k9N1nLRgji7URofe5gBom
  Args:
    query: SELECT t.Name as TrackTitle, a.Name as ArtistName, t.UnitPrice 
FROM Track t
JOIN Album al ON t.AlbumId = al.AlbumId
JOIN Artist a ON al.ArtistId = a.ArtistId
WHERE t.Name = 'So What' AND a.Name = 'Metallica'
================================= Tool Message =================================
Name: execute_sql

[('So What', 'Metallica', 0.99)]
================================== Ai Message ==================================

The track "So What" by Metallica is priced at **$0.99**.
